In [1]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [3]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U "keras>=3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.2/691.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 90.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [4]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [5]:
import keras
import keras_nlp

In [30]:
import json

data = []
with open("/content/Protein_Prompts.txt") as file:
    for line in file:
        try:

            features = json.loads(line)
            template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
            data.append(template.format(**features))

        except json.JSONDecodeError as e:
            # If there's a JSON decode error, skip the line and print the error
            print(f"Skipping line due to error: {e}")

# Limiting the data to first 10 entries
# data


In [31]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

In [32]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,617,270,528 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,617,270,528 (9.75 GB)

 Trainable params: 2,928,640 (11.17 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

In [35]:
# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=50, batch_size=1)

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - loss: 0.6882 - sparse_categorical_accuracy: 0.7938
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - loss: 0.3625 - sparse_categorical_accuracy: 0.8953
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 42s 843ms/step - loss: 0.2713 - sparse_categorical_accuracy: 0.9246
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 42s 830ms/step - loss: 0.2263 - sparse_categorical_accuracy: 0.9448
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 42s 836ms/step - loss: 0.1857 - sparse_categorical_accuracy: 0.9564
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 42s 834ms/step - loss: 0.1687 - sparse_categorical_accuracy: 0.9586
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 42s 836ms/step - loss: 0.1593 - sparse_categorical_accuracy: 0.9594
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 42s 835ms/step - loss: 0.1493 - sparse_categorical_accuracy: 0.9604
Epoch 9/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 42s 836ms/step - loss: 0.1412 - sparse_categorical_accuracy: 0.9611
Epoch 10/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 42s 837ms/st

In [36]:
prompt = template.format(
    instruction="Build a network with 2 layers: a dense layer with 256 units and ReLU activation, followed by a dropout layer with a rate of 0.3.",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Build a network with 2 layers: a dense layer with 256 units and ReLU activation, followed by a dropout layer with a rate of 0.3.

Response:
network: - layer_func: ~tf.layers.dense arguments: name: dense1 inputs: ~layers[-1] units: 256 activation: ~tf.nn.relu - layer_func: ~tf.layers.dropout arguments: name: dropout1 inputs: ~layers[-1] rate: 0.3 training: ~ph_inputs_dict["training"]


In [37]:
prompt = template.format(
    instruction="Create a network with 4 layers: a dense layer with 1024 units and ReLU activation, followed by a dropout layer with a rate of 0.5, a dense layer with 256 units and LeakyReLU activation, and a final dense layer with 64 units and ReLU activation.",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Create a network with 4 layers: a dense layer with 1024 units and ReLU activation, followed by a dropout layer with a rate of 0.5, a dense layer with 256 units and LeakyReLU activation, and a final dense layer with 64 units and ReLU activation.

Response:
network: - layer_func: ~tf.layers.dense arguments: name: dense1 inputs: ~layers[-1] units: 1024 activation: ~tf.nn.relu - layer_func: ~tf.layers.dropout arguments: name: dropout1 inputs: ~layers[-1] rate: 0.5 training: ~ph_inputs_dict["training"] - layer_func: ~tf.layers.dense arguments: name: dense2 inputs: ~layers[-1] units: 256 activation: ~tf.nn.leaky_relu - layer_func: ~tf.layers.dense arguments: name: dense3 inputs: ~layers[-1] units: 64 activation: ~tf.nn.relu
